*This notebook is interactive, and contains both code and text. To run a piece of code, click on the box and hit **Ctrl-Enter**.*

# Regent, a Language for Implicit Parallelism with Sequential Semantics

[Regent](http://regent-lang.org/) is an implicit parallel programming language with sequential semantics. Regent programs look like traditional sequential programs, and you can understand what a program is doing by reading the code top-to-bottom, just like a traditional programming language. Behind the scenes, Regent parallelizes the code, while ensuring that any parallel execution produces results identical to the original sequential execution.

For example, the two recursive calls to `fib` in the code below can automatically run in parallel:

In [ ]:
import "regent"

task fib(i : int) : int
  if i <= 0 then
    return 1
  end

  return fib(i-1) + fib(i-2) -- These calls can run in parallel.
end

task main()
  regentlib.c.printf("fib(5) is %d\n", fib(5))
end
regentlib.start(main)

## Tasks

*Tasks*, like `fib` above, are the fundamental unit of parallelism in Regent. Tasks are just functions: they take arguments, they have a body, and they return a result. Most task arguments, such as argument `i` to `fib`, are passed by-value, and therefore cannot be shared between tasks. Thus the task `fib` is trivially parallel, because it cannot interfere with other tasks.

So far, this is just standard functional programming. However, Regent can also describe imperative programs (where tasks can modify their arguments). For example, the `inc` task below increments every element of its argument `r` by one.

In [ ]:
import "regent"

task inc(r : region(int)) -- Argument r is a container of ints.
where reads writes(r) do  -- Declare how r will be used.
  for x in r do
    @x += 1               -- Dereference each element and increment by 1.
  end
end
  
task sum(r : region(int))
where reads(r) do
  var t = 0
  for x in r do
    t += @x
  end
  return t
end

## Privileges

Ignore the arguments for the moment and focus on the declarations on lines 4 and 11 above. Regent tasks can mutate arguments, but only after requesting *privileges* on said argument. In the case above, `inc` is requesting read-write access to `r`, while `sum` only needs read access. Tasks can only call other tasks with a subset of their own privileges; so `inc` could call `sum`, but `sum` cannot call `inc`.

(Go ahead and try: add a call to `inc(r)` in the body of `sum`. Regent will report an error at the line where you add the call.)

Privileges are the first of a three-part strategy that Regent uses to discover parallelism. Part two has actually been hiding in plain sight, in the arguments on lines 3 and 10. Let's revisit them now.

## Regions

While privileges specify how data is used, regions answer the question of *what* data is being used. Conceptually, regions are containers, like arrays: they contain a number of elements, indexed by a set of keys called an *index space*. (In the example above, the keys were opaque pointers, but they can easily be integers or multi-dimensional points as well.)

However, the analogy only goes so far. Every region is given a unique type; this means that you can't pull a fast one and fool the compiler into thinking you had privileges on something you didn't. For example, the code below will not compile, because `r` and `s` have distinct types.

In [ ]:
import "regent"

task swap(r : region(int), s : region(int))
  r, s = s, r -- ERROR: r and s are different types
end

(Don't worry, there are other ways to swap the contents of regions, if that's what you want.)

Regent uses a [variation on region-based type systems](http://legion.stanford.edu/pdfs/oopsla2013.pdf) to catch these sorts of errors. As a result, a large class of data-race and data-corruption errors are found at compile-time in Regent. (As a bonus, deadlocks are also impossible.)

Having said that, regions are still first-class values. They can be created dynamically, and region arguments might get bound to different regions at different times during a program's execution. For example:

In [ ]:
import "regent"

task f(r : region(int)) -- r will be bound to a different region on each call.
end

task main()
  var s = region(ispace(ptr, 3), int)
  f(s)
  for i = 0, 3 do
    var t = region(ispace(ptr, 5+i), int)
    f(t)
  end
end
regentlib.start(main)

Regions are created with the `region` keyword, which takes an index space and a *field space* (set of fields). So far, all of the field spaces have been simple types like `int`, but structs are typically more common in practice.

Index spaces can built from an opaque index type like `ptr`, as above, or from a structured index type (in 1, 2, 3, etc. dimensions). For example, the following code builds a 1D and a 2D region.

In [ ]:
import "regent"

struct rgba { r : int8, g : int8, b : int8, a : int8 }

task main()
  var line = ispace(int1d, 5, 0) -- 5 element starting at 0
  var grid = ispace(int2d, {x = 4, y = 4}, {x = -1, y = -1}) -- 4x4 block starting at -1x-1

  var vec = region(line, float)
  var img = region(grid, rgba)
end
regentlib.start(main)

One last note about unstructured index spaces. When using the `ptr` type to build an opaque index space, the elements must be allocated individually. This is useful in building incremental data structures like lists, trees, and graphs.

In [ ]:
import "regent"

task main()
  var r = region(ispace(ptr, 5), int) -- Make a region with space for 5 elements.
  var x = new(ptr(int, r))            -- Allocate one element.
  var y = new(ptr(int, r), 3)         -- Allocate a block of 3 more.
end

## Partitions

With regions and partitions, we are already able to exploit [task parallelism](https://en.wikipedia.org/wiki/Task_parallelism) in Regent. Two tasks that use a region read-only will be able to run in parallel, as will tasks that use different regions read-write.

In order to take advantage of [data parallelism](https://en.wikipedia.org/wiki/Data_parallelism), we'll need to use *partitions*. Partitions subdivide a region into multiple pieces, so that multiple tasks can run on those pieces in parallel.

Partitions are not copies of the data, but views. Changes to a subregion inside a partition will be automatically reflected in the parent. This means, for example, that multiple partitions can be used to slice the region's data different ways, and that data will get shuffled around automatically when accessing the different partitions.

There are several ways to create partitions. One straightforward way to create a partition is to assign each element in a region to a color (i.e. in a field of each element), and then pass that coloring to the `partition` operator. Other ways to create partitions are covered in the [Regent language reference](http://regent-lang.org/reference/).

In [ ]:
import "regent"

struct elt { value : int, c0 : int, c1 : int }

-- This task increments each element in a region by the number of elements.
task inc_by_size(r : region(elt))
where reads writes(r) do
  var size = 0
  for x in r do
    size += 1
  end
  for x in r do
    x.value += size
  end
end

task main()
  -- Make a region and name the 4 elements.
  var r = region(ispace(ptr, 4), elt)
  var x0 = new(ptr(elt, r))
  var x1 = new(ptr(elt, r))
  var x2 = new(ptr(elt, r))
  var x3 = new(ptr(elt, r))

  fill(r.value, 0) -- Clear the region.

  -- Make two different colorings of the region.
  x0.c0, x1.c0, x2.c0, x3.c0 = 0, 0, 1, 2
  x0.c1, x1.c1, x2.c1, x3.c1 = 0, 1, 1, 1

  -- Make the two partitions.
  var colors = ispace(int1d, 3) -- Each partition will have 3 subregions.
  var part0 = partition(r.c0, colors)
  var part1 = partition(r.c1, colors)

  -- Call inc_by_size on each partition.
  for color in colors do
    inc_by_size(part0[color])
  end
  for color in colors do
    inc_by_size(part1[color])
  end

  for x in r do
    regentlib.c.printf("element %d has value %d\n", int(x), x.value)
  end
end
regentlib.start(main)

With this we're running in parallel! Each partition is individually disjoint (subregions do not overlap), so each for loop (lines 37-39 and 40-42) can run in parallel. Between (and around) the loops, Regent will handle any data shuffling necessary to ensure that all views onto the data remain consistent.

## Where To From Here

We hope you've enjoyed this introduction to Regent! Feel free to stay here and play around with the code as much as you like. Or, if you'd like to install Regent locally, head over to the [getting started page](http://regent-lang.org/start/) for details.

You might also enjoy reading the [source](https://github.com/StanfordLegion/legion/tree/master/language), [language reference](http://regent-lang.org/reference/), or [paper](http://legion.stanford.edu/pdfs/regent2015.pdf). There is also an [older paper](http://legion.stanford.edu/pdfs/oopsla2013.pdf) which describes a fragment of the type system.

## Credits

Regent is an active research project at Stanford University. More details about the team are available at the [Legion homepage](http://legion.stanford.edu/).